In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import fsolve
from scipy.optimize import root

In [3]:
# 导入基准期变量
N = 30
J = 2

## 2010年的城市部门建设用地面积
K = np.array(pd.read_excel('./data/raw_and_build/urban_constructive_land_area.xlsx').loc[:,'Land2010'])
K = np.delete(K, 25).reshape([N, 1])

## 贸易份额
pi = np.array(pd.read_excel('./data/raw_and_build/trade_share/trade_share_sep.xlsx').iloc[:, 1:]) # 列向加总=1，每一列代表一个进口地区，这里用了不存在跨部门贸易的share

## 流动份额
# m = np.array(pd.read_excel('./data/raw_and_build/migration_share/migration_share_TZ2019.xlsx').iloc[:, 1:]) # 人口普查数据构建的人口流动矩阵，列向加总=1，每一列代表对应一个流出地
m = np.array(pd.read_excel('./data/raw_and_build/migration_share/migration_share_TZ2019.xlsx').iloc[:, 1:]).T # Tombe and Zhu (2019)的数据，列向加总=1，每一列代表对应一个流出地

## 各地区常住居民数量与户籍人口
# L = np.array(pd.read_excel('./data/raw_and_build/macro_var_2010.xlsx').loc[:, 'L']).reshape([N*J, 1])
L_bar = np.array(pd.read_excel('./data/raw_and_build/macro_var_2010.xlsx').loc[:, 'L_bar']).reshape([N*J, 1])
L = m @ L_bar 

## 各地居民的实际收入与价格指数
w = np.array(pd.read_excel('./data/raw_and_build/macro_var_2010.xlsx').loc[:, 'w']).reshape([N*J, 1])
P = np.array(pd.read_excel('./data/raw_and_build/macro_var_2010.xlsx').loc[:, 'P']).reshape([N*J, 1])
W = w / P

In [4]:
# 导入参数
kappa = 1.5 # 流动弹性：使用Tombe and Zhu (2019)的数值
theta = 4 # 贸易弹性：使用Tombe and Zhu (2019)的数值
beta = 0.9 # 居民消费份额：根据2011年《中国统计年鉴》中公布的城镇居民非住房支出份额求得
psi_ag, psi_na = 0.08, 0.92 # 居民最终品消费中农产品和非农产品的份额，根据2010年IO表求得
alpha_ag, alpha_na = 0.28, 0.15 # 劳动产出弹性：根据2010年IO表和部分文献校准值求得
sigma_ag, sigma_na = 0.30, 0.02 # 土地产出弹性：根据2010年IO表和部分文献校准值求得
eta_ag_ag, eta_ag_na = 0.13, 0.05 # 农产品在农村部门和城市部门的产出弹性：根据2010年IO表和部分文献校准值求得
eta_na_ag, eta_na_na = 0.29, 0.78 # 非农产品在农村部门和城市部门的产出弹性：根据2010年IO表和部分文献校准值求得
phiu = 0.1 # 公共支出对居民效用的外部性：使用Leeper et al.(2010)的数值
phip = 0.05 # 公共支出对生产的外部性：使用Leeper et al. (2010)的数值
chi = 0.9 # 公共品拥挤参数：ZC2021，取0.5
eta = 5 # 最终品构成中各中间品的替代弹性：随便取，用不到的
delta = 0 # 体现规模经济效应的规模弹性(基准模型中未体现规模经济效应)
xi_ag, xi_na = 0.01, 0.99 # 政府购买的最终产品份额：根据2010年IO表直接求得
t = 1 # 收入税：暂时先假设的

In [5]:
# 求解基准均衡
R = np.ndarray([60, 1])
X0 = np.zeros([60, 1])
X = np.ones([60, 1])
iter_num = 0

while abs((X0 - X)).sum() > 1e-10:
    X0 = X
    R = pi @ X0
    wL = (R.reshape([30, 2]) * np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1])
    wL = wL / wL.sum() # 标准化
    vL = (wL.reshape([30, 2]) * np.array([(alpha_ag + sigma_ag) / (beta + t - beta * t) / alpha_ag, 1]).reshape([1, 2])).reshape([60, 1])
    PG = (wL.reshape([30, 2]) * np.array([t * (alpha_ag + sigma_ag) / (beta + t - beta * t) / alpha_ag, 1 - beta + sigma_na / alpha_na + beta * t]).reshape([1, 2])).reshape([60, 1])
    A = (beta * (1 - t) * np.repeat(vL.reshape([30, 2]).sum(axis=1).reshape([30, 1]), 2, axis=1) * np.array([psi_ag, psi_na]).reshape([1, 2])).reshape([60, 1])
    C = (np.repeat(PG.reshape([30, 2]).sum(axis=1).reshape([30, 1]), 2, axis=1) * np.array([xi_ag, xi_na]).reshape([1, 2])).reshape([60, 1])
    B_ag = ((R.reshape([30, 2]) * np.array([eta_ag_ag, eta_ag_na]).reshape([1, 2])).sum(axis=1)).reshape([30, 1])
    B_na = ((R.reshape([30, 2]) * np.array([eta_na_ag, eta_na_na]).reshape([1, 2])).sum(axis=1)).reshape([30, 1])
    B = np.concatenate([B_ag, B_na], axis=1).reshape([60, 1])
    X = A + B + C

    iter_num += 1
    print(str(iter_num), end='\r')

equilibrium_dict = {
    'pi' : pi,
    'm' : m,
    'L' : L,
    'L_bar' : L_bar,
    'w' : w,
    'P' : P,
    'W' : W,
    'R' : R,
    'X' : X,
    'K' : K
}

parameter = {
    'kappa' : kappa,
    'theta' : theta,
    'beta' : beta,
    'psi_ag' : psi_ag,
    'psi_na' : psi_na,
    'alpha_ag' : alpha_ag,
    'alpha_na' : alpha_na,
    'sigma_ag' : sigma_ag,
    'sigma_na' : sigma_na,
    'eta_ag_ag' : eta_ag_ag, 
    'eta_ag_na' : eta_ag_na,
    'eta_na_ag' : eta_na_ag, 
    'eta_na_na' : eta_na_na,
    'phiu' : phiu,
    'phip' : phip,
    'chi' : chi,
    'eta' : eta,
    'delta' : delta,
    'xi_ag' : xi_ag,
    'xi_na' : xi_na,
    't' : t
}
np.savez('./data/Equilibrium2010.npz', **equilibrium_dict)
np.savez('./data/Parameter.npz', **parameter)

In [6]:
# 求解基准均衡(用现成package的求解函数做验证)

def SolveCounter(X):
    X = X.reshape([60, 1])
    R = pi @ X
    wL = (R.reshape([30, 2]) * np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1])
    wL = wL / wL.sum() # 标准化
    vL = (wL.reshape([30, 2]) * np.array([(alpha_ag + sigma_ag) / (beta + t - beta * t) / alpha_ag, 1]).reshape([1, 2])).reshape([60, 1])
    PG = (wL.reshape([30, 2]) * np.array([t * (alpha_ag + sigma_ag) / (beta + t - beta * t) / alpha_ag, 1 - beta + sigma_na / alpha_na + beta * t]).reshape([1, 2])).reshape([60, 1])
    A = (beta * (1 - t) * np.repeat(vL.reshape([30, 2]).sum(axis=1).reshape([30, 1]), 2, axis=1) * np.array([psi_ag, psi_na]).reshape([1, 2])).reshape([60, 1])
    C = (np.repeat(PG.reshape([30, 2]).sum(axis=1).reshape([30, 1]), 2, axis=1) * np.array([xi_ag, xi_na]).reshape([1, 2])).reshape([60, 1])
    B_ag = ((R.reshape([30, 2]) * np.array([eta_ag_ag, eta_ag_na]).reshape([1, 2])).sum(axis=1)).reshape([30, 1])
    B_na = ((R.reshape([30, 2]) * np.array([eta_na_ag, eta_na_na]).reshape([1, 2])).sum(axis=1)).reshape([30, 1])
    B = np.concatenate([B_ag, B_na], axis=1).reshape([60, 1])
    
    return (X - A - B - C).flatten()

X_test = root(SolveCounter, x0=np.ones(60))
X_test = X_test.x.reshape([60, 1])
print(abs(X - X_test).sum() < 0.000001) # 检查

True
